In [29]:
import cv2
import numpy as np
import math
import io
import os
import threading
from PIL import Image, ImageDraw
from scipy import ndimage
from matplotlib import pyplot as plt
import glob
import zmq
#import import_ipynb
#import canny_non_max_suppression as canny_nms

In [30]:
def nothing(x):
  pass

DEFAULT_MASKNAME = "./processed/mask_original.png"
MASK_NAME = DEFAULT_MASKNAME


In [31]:
def get_spotmask(r = 3):
    dst = np.full([r*2 + 1, r*2 + 1], 0, dtype=np.uint8)
    if (r > 0):
        img = Image.fromarray(dst, mode='L')
        draw = ImageDraw.Draw(img)
        draw.ellipse((0, 0, 2*r+1, 2*r+1), fill = (255))
        dst = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0])
    else:
        dst[0, 0] = 255
    return dst

def get_circlemask(r = 3):
    dst = np.full([r*2 + 1, r*2 + 1], 0, dtype=np.uint8)
    if (r > 0):
        img = Image.fromarray(dst, mode='L')
        draw = ImageDraw.Draw(img)
        draw.ellipse((0, 0, 2*r+1, 2*r+1), outline = (255))
        dst = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0])
    else:
        dst[0, 0] = 255
    return dst

def fill_mask_gray(src, mask, idx_y, idx_x, value = 255):
    rows, cols = src.shape
    mrows, mcols = mask.shape
    valid_coord = True
    valid_coord &= (idx_x - mcols//2) > 0
    valid_coord &= (idx_y - mrows//2) > 0
    valid_coord &= (idx_x + mcols//2) < cols
    valid_coord &= (idx_y + mrows//2) < rows
    if (valid_coord):
        center_x = mcols//2
        center_y = mrows//2
        for row in range(mrows):
            for col in range(mcols):
                crd_x = idx_x - center_x + col
                crd_y = idx_y - center_y + row 
                if (mask[row, col] != 0):
                    src[crd_y, crd_x] = value
    return src

def test_window_maskcross(src, mask, idx_x, idx_y):
    rows, cols, _ = src.shape
    mrows, mcols = mask.shape
    valid_coord = True
    wnd = np.zeros([mrows, mcols, 3], dtype=np.uint8)
    valid_coord &= (idx_x - mcols//2) > 0
    valid_coord &= (idx_y - mrows//2) > 0
    valid_coord &= (idx_x + mcols//2) < cols
    valid_coord &= (idx_y + mrows//2) < rows
    is_crossed = False
    bound_color = (255, 255, 255)
    if (valid_coord):
        center_x = mcols//2
        center_y = mrows//2
        for row in range(mrows):
            for col in range(mcols):
                if (mask[row, col] != 0):
                    crd_x = idx_x - center_x + col
                    crd_y = idx_y - center_y + row 
                    if ((src[crd_y, crd_x, 0] ==  bound_color[0]) and
                        (src[crd_y, crd_x, 1] ==  bound_color[1]) and
                        (src[crd_y, crd_x, 2] ==  bound_color[2])):
                        is_crossed = True
    return is_crossed


def fill_mask3(src, mask, idx_y, idx_x, color = (0, 255, 0)):
    rows, cols,_ = src.shape
    mrows, mcols = mask.shape
    valid_coord = True
    valid_coord &= (idx_x - mcols//2) > 0
    valid_coord &= (idx_y - mrows//2) > 0
    valid_coord &= (idx_x + mcols//2) < cols
    valid_coord &= (idx_y + mrows//2) < rows
    if (valid_coord):
        center_x = mcols//2
        center_y = mrows//2
        for row in range(mrows):
            for col in range(mcols):
                crd_x = idx_x - center_x + col
                crd_y = idx_y - center_y + row 
                if (mask[row, col] != 0):
                    src[crd_y, crd_x, 0] = color[0]
                    src[crd_y, crd_x, 1] = color[1]
                    src[crd_y, crd_x, 2] = color[2]
    return src

def flood_window(src, mask, spotmask, idx_x, idx_y, color = (0, 255, 0) ):
    rows, cols, _ = src.shape
    mrows, mcols = spotmask.shape
    center_x = mcols//2
    center_y = mrows//2
    started = False
    def flood_step(src, mask,  started, row, col, 
                   idx_x_ = idx_x, idx_y_ = idx_y, 
                   center_x_ = center_x, 
                   center_y_ = center_y, color_ = color):
        #global started#, center_x, center_y
        #global idx_y, idx_x, color
        crd_x = idx_x_ - center_x_ + col
        crd_y = idx_y_ - center_y_ + row 
        if ((src[crd_y, crd_x, 0] == color_[0]) and
            (src[crd_y, crd_x, 1] == color_[1]) and
            (src[crd_y, crd_x, 2] == color_[2])):
            started = True
        if ((src[crd_y, crd_x, 0] == 255) and
            (src[crd_y, crd_x, 1] == 255) and
            (src[crd_y, crd_x, 2] == 255)):
            started = False
        if (started):
            src[crd_y, crd_x, 0] = color[0]
            src[crd_y, crd_x, 1] = color[1]
            src[crd_y, crd_x, 2] = color[2]
            mask[crd_y, crd_x] = 255
        return src, mask, started
    valid_coord = (idx_x - mcols//2) > 0
    valid_coord &= (idx_y - mrows//2) > 0
    valid_coord &= (idx_x + mcols//2) < cols
    valid_coord &= (idx_y + mrows//2) < rows
    if (valid_coord):
        for row in range(mrows):
            started = False
            for col in range(mcols):
                src, mask, started = flood_step(src, mask, started, row, col)
        for row in range(mrows):
            started = False
            for col in range(mcols, -1, -1):
                src, mask, started = flood_step(src, mask, started, row, col)
        for col in range(mcols):
            started = False
            for row in range(mrows):
                src, mask, started = flood_step(src, mask, started, row, col)
        for col in range(mcols):
            started = False
            for row in range(mrows, -1, -1):
                src, mask, started = flood_step(src, mask, started, row, col)

    return src, mask

def highlight_spot(src, mask, spotmask, x = -1, y = -1, color = (0, 255, 0)):
    if (src.ndim == 2):
        rows, cols = src.shape
        dst = np.full([rows, cols, 3], 0, dtype=np.uint8)
        #dst_mask = np.full([rows, cols, 3], 0, dtype=np.uint8)
        #dst_cmask = np.full([rows, cols, 3], 0, dtype=np.uint8)
        for row in range(rows):
            for col in range(cols):
                dst[row, col, 0] = src[row, col]
                dst[row, col, 1] = src[row, col]
                dst[row, col, 2] = src[row, col]
    else:
        rows, cols, _ = src.shape
        dst = src
        #dst_cmask = np.full([rows, cols, 3], 0, dtype=np.uint8)
        rows, cols, _ = src.shape
    #img = Image.fromarray(mask, mode='L')
    #draw = ImageDraw.Draw(img)
    #if ((x -r > 0) & (y - r > 0) & (x + r < cols) & (y + r < rows)):
    #    draw.ellipse((x - r, y - r, x + r, y + r), fill = (255) , outline = (255))
    #dst_mask = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0])
    gray_color = max(color) 
    dst_mask = fill_mask_gray(mask, spotmask, y, x, gray_color)
    #img = Image.fromarray(dst, mode='RGB')
    #draw = ImageDraw.Draw(img)
    #if ((x -r > 0) & (y - r > 0) & (x + r < cols) & (y + r < rows)):
    #    draw.ellipse((x - r, y - r, x + r, y + r), fill = color , outline = color)
    #dst = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
    dst = fill_mask3(dst, spotmask, y, x, color=color)
    
    return dst, dst_mask

def apply_mask(src, mask, color = (0, 255, 0)):
    if (src.ndim == 2):
        rows, cols = src.shape
        dst = np.full([rows, cols, 3], 0, dtype=np.uint8)
        #dst_mask = np.full([rows, cols, 3], 0, dtype=np.uint8)
        dst_cmask = np.full([rows, cols, 3], 0, dtype=np.uint8)
        for row in range(rows):
            for col in range(cols):
                dst[row, col, 0] = src[row, col]
                dst[row, col, 1] = src[row, col]
                dst[row, col, 2] = src[row, col]
    else:
        rows, cols, _ = src.shape
        dst = src
        dst_cmask = np.full([rows, cols, 3], 0, dtype=np.uint8)
        rows, cols, _ = src.shape
    for row in range(rows):
        for col in range(cols):
            if (mask[row, col] != 0):
                dst_cmask[row, col, 0] = color[0]
                dst_cmask[row, col, 1] = color[1]
                dst_cmask[row, col, 2] = color[2]
            else:
                dst_cmask[row, col, 0] = dst[row, col, 0]
                dst_cmask[row, col, 1] = dst[row, col, 1]
                dst_cmask[row, col, 2] = dst[row, col, 2]
    result = dst_cmask
    return result


def highlight_circle(src, x = -1, y = -1, r = 3):
    if (src.ndim == 2):
        rows, cols = src.shape
        dst = np.full([rows, cols, 3], 0, dtype=np.uint8)
        for row in range(rows):
            for col in range(cols):
                dst[row, col, 0] = src[row, col]
                dst[row, col, 1] = src[row, col]
                dst[row, col, 2] = src[row, col]
    else:
        rows, cols, _ = src.shape
        dst = src
        rows, cols, _ = src.shape
    img = Image.fromarray(dst, mode='RGB')
    draw = ImageDraw.Draw(img)
    if ((x -r > 0) & (y - r > 0) & (x + r < cols) & (y + r < rows)):
        draw.ellipse((x - r, y - r, x + r, y + r),  outline = (255, 0, 0))
    dst = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
    return dst


def highlight_vmarker(src, angle = np.pi/2, dxc = 0):
    if (src.ndim == 2):
        rows, cols = src.shape
        dst = np.full([rows, cols, 3], 0, dtype=np.uint8)
        for row in range(rows):
            for col in range(cols):
                dst[row, col, 0] = src[row, col]
                dst[row, col, 1] = src[row, col]
                dst[row, col, 2] = src[row, col]
    else:
        dst = src
        rows, cols, _ = src.shape
    x_bot = np.int16((cols/2.0) - (rows/2.0)*np.tan((np.pi/2.0) - angle)) + dxc
    x_top = np.int16((cols/2.0) + (rows/2.0)*np.tan((np.pi/2.0) - angle)) + dxc
    img = Image.fromarray(dst, mode='RGB')
    draw = ImageDraw.Draw(img)
    
    draw.line((x_top, 0, x_bot, rows - 1), fill=(255,0,0), width=2)
    result = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
    return result

def highlight_line(src, line, fill = (0 ,0, 255)):
    width = line[4]
    if (src.ndim == 2):
        rows, cols = src.shape
        dst = np.full([rows, cols, 3], 0, dtype=np.uint8)
        for row in range(rows):
            for col in range(cols):
                dst[row, col, 0] = src[row, col]
                dst[row, col, 1] = src[row, col]
                dst[row, col, 2] = src[row, col]
    else:
        dst = src
        rows, cols, _ = src.shape
    img = Image.fromarray(dst, mode='RGB')
    draw = ImageDraw.Draw(img)
    x1 = line[0]; 
    y1 = line[1]; 
    x2 = line[2]; 
    y2 = line[3]
    draw.line((x1, y1, x2, y2), fill=fill, width=width)
    dst = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
    return dst

def highlight_linemask(src, line):
    width = line[4]
    if (src.ndim == 2):
        rows, cols = src.shape
        dst = src
    else:
        rows, cols,_ = src.shape
        dst = np.full([rows, cols], 0, dtype=np.uint8)

    img = Image.fromarray(dst, mode='L')
    draw = ImageDraw.Draw(img)
    x1 = line[0]; 
    y1 = line[1]; 
    x2 = line[2]; 
    y2 = line[3]
    draw.line((x1, y1, x2, y2), fill=(255), width=width)
    dst = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0])
    return dst

def highlight_lineset(src, lineset):
    dst = np.copy(src)
    for line in  lineset:
        dst = highlight_line(dst, line, (0 ,255, 0))
    return dst

def highlight_linemaskset(src, lineset):
    dst = np.zeros_like(src)
    for line in  lineset:
        dst = highlight_linemask(dst, line)
    return dst

def highlight_roi(src, x1, y1, w, h):
    rows, cols = src.shape
    dst = np.full([rows, cols, 3], 0, dtype=np.uint8)
    for row in range(rows):
        for col in range(cols):
            dst[row, col, 0] = src[row, col]
            dst[row, col, 1] = src[row, col]
            dst[row, col, 2] = src[row, col]
    img = Image.fromarray(dst, mode='RGB')
    draw = ImageDraw.Draw(img)
    draw.rectangle((x1, y1, x1 + w, y1 + h), outline=(255,0,0))
    result = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
    return result

def save_matrix(M, filename='dump.txt'):
    np.savetxt(fname=filename, X=M, fmt="%d", delimiter=" ", newline='\n')
    return

mouse_down = False
x_coord = -1
y_coord = -1
x_lcoord = -1
y_lcoord = -1
def onMouseCallBack(event, x, y, flags, param):
    global mouse_down
    global x_coord, y_coord, x_lcoord, y_lcoord
    x_coord = x - 5
    y_coord = y - 5
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_down = True
        x_lcoord = x
        y_lcoord = y
    if event == cv2.EVENT_LBUTTONUP and mouse_down:
        mouse_down = False
    return None

def load_image(img_name):
    img_src = cv2.imread(img_name)
    if (len(img_src.shape) > 2):
        img_dst = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
    else:
        img_dst = img
    print(type(img_src), img_src.shape, img_dst.shape)
    return img_dst


In [32]:
def get_arctan(dY, dX):
    anglepi = math.atan2(dY, dX) # -pi; +pi
    angle = np.round((180.0/np.pi)*anglepi)
    if (angle < 0):
        angle = 180 + angle
    return np.uint16(angle)

def test_line_lng(x1, y1, x2, y2, w):
    is_valid = False
    if ((abs(x2 - x1) > w) & (abs(y2 - y1) > w)):
        is_valid = True
    return is_valid

def save_line_list(llist, fname):
    str1 = fname.split('photo')[1]
    filename = "../processed/histogramB_test" + str1[0] + ".txt"
    with open(filename, "w") as dst:
        dst.write('ind, freq angle\n')
        for line in llist:
            str1 = "{} {} {}".format(*line) + '\n'
            dst.write(str1)
            
def save_mask(fname, mask):
    global MASK_NAME, DEFAULT_MASKNAME
    img = Image.fromarray(mask, 'L')
    #str1 = fname.split('photo')[1]
    newname = MASK_NAME
    print(newname)
    img.save(newname)

file_idx = 0
width = 15
quit = False

In [33]:
cv2.namedWindow('Canny')
#if (os.path.isfile("./DatasetNavigator/DatasetNavigator")):
#    os.startfile("./DatasetNavigator/DatasetNavigator")

hh='Max'
hl='Min'
wnd = "Gray"


# Load of images
namemask = './processed/bright_photo*.png'
#imglist = []
#name_list = []
#for image_name in glob.glob(namemask):
#    img_tmp = load_image(image_name)
#    #img_tmp = cv2.resize(img_tmp, dsize=(0,0), fx=0.5, fy=0.5)
#    imglist.append(img_tmp)
#    name_list.append(os.path.basename(image_name))
original_image = np.zeros([320, 640], dtype=np.uint8)
original_name = "./original_photo.png"

file_num = 2#len(imglist)
if (file_num > 0):
    cv2.createTrackbar("file#", "Canny",0, file_num - 1,nothing)
    cv2.createTrackbar("width#", "Canny",0, 15,nothing)
    cv2.createTrackbar("min#", "Canny",0, 255,nothing)
    cv2.createTrackbar("max#", "Canny",0, 255,nothing)
    cv2.createTrackbar("tool#", "Canny",0, 25,nothing)
file_idx = cv2.getTrackbarPos("file#", "Canny")
cv2.setWindowTitle("Canny", os.path.basename(original_name))
width = 3#cv2.getTrackbarPos("width#", "Canny")
lowcanny = cv2.getTrackbarPos("min#", "Canny")
highcanny = cv2.getTrackbarPos("max#", "Canny")
toolnumber = cv2.getTrackbarPos("tool#", "Canny")
#img_gray = np.copy(imglist[file_idx])

#img_mask = np.zeros_like(original_image)
COLCENTER1 = original_image.shape[1]//2
ROWCENTER1 = original_image.shape[0]//2
spot_mask = get_spotmask(width)

In [34]:
#img_tmp = img_gray
#
img_initimage = np.copy(original_image)
img_finalimage = np.copy(original_image)

In [ ]:
hard_mask = np.zeros_like(original_image)
img_mask = np.zeros_like(original_image)
trigger = True
trigger2 = True
trigger3 = False
oldlow = lowcanny
oldhigh = highcanny
oldwidth = width
view_number = 0

def process_highgui():
    global file_idx ,trigger, trigger2,  imglist
    global hard_mask, width, oldwidth, lowcanny, highcanny
    global oldhigh, oldlow, img_initimage, img_finalimage
    global spot_mask, toolnumber, view_numberi, MASK_NAME, DEFAULT_MASKNAME
    global original_name, original_image
    # Prepare our context and sockets
    context = zmq.Context()

    # Connect to weather server
    subscriber = context.socket(zmq.SUB)
    isconnected = subscriber.connect("tcp://127.0.0.1:5561")
    isconnected = True
    if (isconnected):
        sndhwm = 2048;
        #zmq.setsockopt(m_publisher, ZMQ_SNDHWM, sndhwm);
        subscriber.set_hwm(sndhwm)
        #subscriber.setsockopt(zmq.SUBSCRIBE, b"10001")
        subscriber.setsockopt(zmq.SUBSCRIBE, b"")
        subscriber.setsockopt(zmq.LINGER, 0)  # All topics
        subscriber.RCVTIMEO = 500  # timeout: 1 sec
        # Initialize poll set
        poller = zmq.Poller()
        #poller.register(receiver, zmq.POLLIN)
        poller.register(subscriber, zmq.POLLIN)
        print("connected")
    else:
        print("not connected")
    while(not quit):
        if (isconnected):
            try:
                socks = dict(poller.poll(200))
            except KeyboardInterrupt:
                print("Ex1")
                break
            if subscriber in socks:
                message = subscriber.recv_string()
                zmq_orig_name = ""
                print("message0:",message)
                if (message):
                    #print("Hello")
                    name_tuple = message[:].split('*')
                    zmq_orig_name = name_tuple[0]
                    mask_found = False
                    if (len(name_tuple) > 1):
                        if (name_tuple[1]):
                            mask_found = True
                    if (mask_found):
                        mname = name_tuple[1]
                        tmppath = os.path.dirname(name_tuple[0])
                        tmppath ,_ =  os.path.split(tmppath)
                        MASK_NAME = tmppath + "/masks/"  + mname
                        if (os.path.isfile(MASK_NAME)):
                            hard_mask = load_image(MASK_NAME)
                        else:
                            hard_mask = np.zeros_like(original_image)
                        print(MASK_NAME)
                        img_mask = np.zeros_like(original_image)
                    else:
                        print("mask not found")
                        if (name_tuple[0]):
                            print("tuple:",name_tuple)
                            tmppath = os.path.dirname(name_tuple[0])
                            tmppath ,_ =  os.path.split(tmppath)
                            bname = os.path.basename(zmq_orig_name)
                            suffix = bname.split('photo')[1]
                            MASK_NAME = tmppath + "/masks/mask_photo" + suffix
                            hard_mask = np.zeros_like(original_image)
                            img_mask = np.zeros_like(original_image)
                        else:
                            print("message:",message)
                
                #os.path.isfile(fname) 
                if (os.path.isfile(zmq_orig_name)):
                    bname = os.path.basename(zmq_orig_name)
                    img_tmp = load_image(zmq_orig_name)
                    original_image = img_tmp
                    original_name = zmq_orig_name
                    trigger = True
        file_idx = cv2.getTrackbarPos("file#", "Canny")
        width = cv2.getTrackbarPos("width#", "Canny")
        lowcanny = cv2.getTrackbarPos("min#", "Canny")
        highcanny = cv2.getTrackbarPos("max#", "Canny")
        toolnumber = cv2.getTrackbarPos("tool#", "Canny")
        if (trigger):
            oldhigh = highcanny
            oldlow = lowcanny
            cv2.setWindowTitle("Canny", os.path.basename(original_name))
            #img_gray = np.copy(imglist[file_idx])
            vnumber = view_number
            if (vnumber == 0):
                img_canny = cv2.Canny(original_image,
                     threshold1=lowcanny, 
                     threshold2=highcanny, 
                     apertureSize=3)
                img_initimage  = apply_mask(img_canny, hard_mask, color = (0, 255, 0))
            elif (vnumber == 1):
                img_initimage  = apply_mask(original_image, hard_mask, color = (0, 255, 0))
            trigger = False
            trigger2 = True
        if (quit): break
            
def draw_spot():           
    global file_idx, mouse_down, trigger, trigger2
    global trigger3, x_coord, x_lcoord, y_coord
    global y_lcoord, width, hard_mask, img_mask
    global quit, width, lowcanny, highcanny
    global oldhigh, oldlow, img_initimage, img_finalimage
    global spot_mask, toolnumber
    while(not quit):
        if (mouse_down):
            if (not trigger): 
                if (toolnumber == 2): 
                    img_finalimage, img_mask = highlight_spot(img_finalimage, img_mask, 
                                                         spot_mask, x_coord, y_coord, 
                                                         color = (0, 0, 255))
                    trigger3 = True
                elif (toolnumber == 1):
                    x_froz = int(x_coord)
                    y_froz = int(y_coord)
                    is_allowed = not test_window_maskcross(img_finalimage, spot_mask, 
                                                      x_froz, y_froz)
                    if (is_allowed):
                        img_finalimage, img_mask = highlight_spot(img_finalimage, 
                                                        img_mask, spot_mask, 
                                                        x_froz, y_froz, 
                                                        color = (0, 0, 255))
                        trigger3 = True
                elif (toolnumber == 3): 
                    img_finalimage, img_mask = flood_window(img_finalimage, 
                                                            img_mask, spot_mask,
                                                            x_coord, y_coord, 
                                                            color = (0, 0, 255) )
                    trigger3 = True
                elif (toolnumber == 4): 
                    x_froz = int(x_coord)
                    y_froz = int(y_coord)
                    is_allowed = not test_window_maskcross(img_finalimage, spot_mask, 
                                                      x_froz, y_froz)
                    if (is_allowed):
                        img_finalimage, _ = highlight_spot(img_finalimage, img_mask, 
                                                         spot_mask, x_froz, y_froz, 
                                                         color = (0, 0, 0))
                    trigger3 = True
            
    
def main_loop():
    global file_idx
    global mouse_down
    global trigger
    global trigger2
    global trigger3
    global x_coord
    global x_lcoord
    global y_coord
    global y_lcoord
    global width
    global oldwidth
    global hard_mask
    global img_mask
    global quit
    global width
    global lowcanny
    global highcanny
    global oldhigh
    global oldlow
    global img_initimage
    global img_finalimage
    global spot_mask
    global view_number
    global original_name, original_image
    cv2.setMouseCallback("Canny",onMouseCallBack)
    t1 = threading.Thread(target=process_highgui, args=())
    t2 = threading.Thread(target=draw_spot, args=())
    t1.start()
    t2.start()
    #t2.start()
    old_fidx = file_idx
    circle_mask = get_circlemask(width)
    while(not quit):
        if (oldwidth != width):
            oldwidth = width
            spot_mask = get_spotmask(width)
            circle_mask = get_circlemask(width)
        if (file_idx != old_fidx):
            old_fidx = file_idx
            #trigger = True
        if ((oldhigh != highcanny) or (oldlow != lowcanny)):
            trigger = True
        
        if (trigger3):
            trigger3 = False
            #img_finalimage2 = fill_mask3(img_finalimage, circle_mask, y_coord, x_coord)
            #cv2.imshow("Canny",img_finalimage2)

            
        if (trigger2):
            img_finalimage, img_mask = highlight_spot(img_initimage, img_mask, 
                                                     spot_mask,
                                                     color = (0, 0, 255))
            trigger2 = False
        img_finalimage2 = np.copy(img_finalimage)
        img_finalimage2 = fill_mask3(img_finalimage2, circle_mask, 
                                     y_coord, x_coord,
                                     color=(255,0,0))
        cv2.imshow("Canny",img_finalimage2)


        k = cv2.waitKey(10) & 0xFF
        if k == ord('v'):
                img_mask = np.zeros_like(original_image)
                view_number = (view_number + 1)%2
                trigger = True
        elif k == ord('c'):
                img_mask = np.zeros_like(original_image)
                trigger = True
        elif k == ord('s'):
                hard_mask = cv2.bitwise_or(hard_mask, img_mask)
                img_mask = np.zeros_like(original_image)
                trigger = True
        elif k == ord('f'):
            save_mask(original_name, hard_mask)
            cv2.setWindowTitle("Canny", os.path.basename(original_name) + " (saved)")
            print("file is saved")
            k = cv2.waitKey(10) & 0xFF
        elif k == ord('r'):
            hard_mask = np.zeros_like(original_image)
            img_mask = np.zeros_like(original_image)
            trigger = True
        elif k == 27:
            quit = True
            t1.join()
            t2.join()
            break
main_loop()
#
cv2.destroyAllWindows()

connected
message0: /home/user/MySoftware/foreign code/netology_JN/Diplom/Datasets/custom_roadlane/normalized/bright_photo0.png*mask_photo0.png
<class 'numpy.ndarray'> (320, 640, 3) (320, 640)
/home/user/MySoftware/foreign code/netology_JN/Diplom/Datasets/custom_roadlane/masks/mask_photo0.png
<class 'numpy.ndarray'> (192, 512, 3) (192, 512)
/home/user/MySoftware/foreign code/netology_JN/Diplom/Datasets/custom_roadlane/masks/mask_photo0.png
file is saved
